In [1]:
import pandas as pd
import xlsxwriter
import numpy as np
from datetime import datetime,timedelta

In [2]:
date1 = '190606'  # 文件名日期
date2 = '2019-06-06'
date3 = '190605'

In [3]:
inputPath = "G:/work/daily/DataShow/190606/"  # 输出路径
inputPath2 = "G:/work/daily/DataShow/"

In [4]:
def laidan(x):
    if x is np.nan:
        return 0
    else:
        return 1

In [5]:
def fahuo(a,b,c):
    if a is not np.nan:
        return 1
    elif a is np.nan and b == "交易完成":
        return 1
    elif a is np.nan and c is not np.nan:
        return 1
    else:
        return 0

In [6]:
def qianshou(a,b):
    if a is not np.nan:
        return 1
    elif a is np.nan and b == "交易完成":
        return 1
    else:
        return 0

In [7]:
def jihuo(x):
    if x == "交易完成":
        return 1
    else:
        return 0

In [8]:
def paidan(x):
    if x is np.nan:
        return 0
    else:
        return 1

In [9]:
inputPath = "G:/work/daily/DataShow/190606/"  # 输出路径

In [10]:
data1 = pd.read_csv(inputPath + "df1.csv")


d:\python\envs\jupyterpy36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,25,30,35,48,60) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df1 = data1.copy()

In [12]:
df1.columns

Index(['订单编号', '原始订单', '订单类型', '订单状态', '订单子状态', '取消原因', '订单挂起原因', '安装失败原因',
       '订单来源', '结算状态', '支付平台', '机价款', '店铺', 'AB类型', '拆单类型', '制式类型', '订单金额',
       '支付方式', '支付流水号', '收取卡密号码', '订单生成时间', '支付完成时间', '交易完成时间', '发货时间',
       '物流单回填时间', '用户名', 'QQ号', '入网用户姓名', '入网身份证号', '入网手机号', '联系人电话号码',
       'ICCID', '联系住址', '收货人姓名', '收货地址', '收货人电话号码', '收货人邮箱', '邮编',
       '所在省 / 市 / 县', '入网人所在省/市/县', '号码归属地', '配送方式', '营业厅地址', '配送时间', '优惠劵',
       '优惠券编码', 'F码', 'F码名称', '产品推荐人', 'CPS推荐人', '订单备注', '物流单号', '物流签收时间',
       '承运商', '是否上传签收证明项', '预存款发票类别', '物品费发票抬头', '物品费抬头名称', '发票内容', '企业税号',
       '用户留言', '大小卡类型', '写卡渠道', '卡号', '串号', '销售品编号', '销售品名称', '销售品类型', '销售品数量',
       '销售品价格', '合作商', '实际分账金额', '套餐', '主号码', '副号码', '现金预存款', '可选包', '合约补贴',
       '其他', '靓号低消', '靓号等级', '靓号预存款', '营业厅送货方式', '是否线下模式', '是否线下转线上',
       '线下转线上原因', '营业厅送货iccid', '身份证图片1', '身份证图片2', '身份证图片3', '身份证图片4',
       '实名信息复核中时间', '销售品文本步骤名称', '用户选择的步骤内容', '京东不可达原因', '订单生成日期', '订单生成小时'],
      dtype='object')

In [13]:
split1 = pd.DataFrame((x.split(' ') for x in df1['订单生成时间']),index=df1.index,columns=['订单生成日期','订单生成小时'])
df1 = pd.merge(df1, split1, left_index=True, right_index=True)

In [14]:
data2 = pd.read_excel(inputPath + "3 7日前新生产导出.xlsx")
split2 = pd.DataFrame((x.split(' ') for x in data2['订单生成时间']),index=data2.index,columns=['订单生成日期','订单生成小时'])
data2 = pd.merge(data2, split2, left_index=True, right_index=True)

In [15]:
data5 = pd.read_excel(inputPath + "3 7日京东平台导出.xlsx")
split5 = pd.DataFrame((x.split(' ') for x in data5['用户下单时间']),index=data5.index,columns=['用户下单日期','用户下单小时'])
data5 = pd.merge(data5, split5, left_index=True, right_index=True)

In [16]:
san_days = (datetime.strptime(date2, '%Y-%m-%d') - timedelta(days=3)).strftime('%Y-%m-%d')
qi_days  = (datetime.strptime(date2, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y-%m-%d')

In [17]:
df3 = pd.read_excel(inputPath2 + "首充明细"+ date3+".xlsx", sheet_name="Sheet1") # 首充历史表
df4 = pd.read_excel(inputPath + "首充新增" + date1 + ".xlsx") # 当日历史表
df4 = df4[["用户名"]]

In [18]:
df1_3ri = data1[data1["订单生成日期"] == san_days]
df2_3ri = data2[data2["订单生成日期"] == san_days]
df5_3ri = data5[data5["用户下单日期"] == san_days] # 京东表

In [19]:
df1_3ri = df1_3ri[(True ^ df1_3ri['订单状态'].isin(['办理成功', '办理中', '办理失败']))]
df1_3ri = df1_3ri[pd.notnull(df1_3ri['所在省 / 市 / 县'])]
df1_3ri = df1_3ri[df1_3ri["订单编号"].str.contains('订单编号') == False]
df1_3ri = df1_3ri.drop_duplicates(subset=['订单编号'], keep='first')
df1_3ri = df1_3ri[pd.notnull(df1_3ri['号码归属地'])]
df1_3ri = df1_3ri[pd.notnull(df1_3ri['入网手机号'])]
split1 = pd.DataFrame((x.split(' ') for x in df1_3ri['订单生成时间']),index=df1_3ri.index,columns=['订单生成日期','订单生成小时'])
df1_3ri = pd.merge(df1_3ri, split1, left_index=True, right_index=True)

In [20]:
df2_3ri = df2_3ri[df2_3ri["模式分类"] == "京东模式"]
df2_3ri = df2_3ri[pd.notnull(df2_3ri['是否下省'])]

In [21]:
df34 = pd.concat([df3, df4],sort=False)
df34 = df34.drop_duplicates(subset=['用户名'], keep='first')
# df34 = df34[df34["订单编号"].str.contains('订单编号') == False]
df34['用户名'] = df34['用户名'].map(lambda x: round(x, 0))

In [22]:
df51 = df5_3ri[["运营商单号", "物流单号"]]
df52 = df5_3ri[["运营商单号", "APP操作时间"]]
df51 = df51[pd.notnull(df51['物流单号'])]
df52 = df52[pd.notnull(df52['APP操作时间'])]

In [23]:
df11 = pd.merge(df1_3ri, df2_3ri[["订单编号", "模式分类", "是否下省"]],
                    left_on="订单编号", right_on="订单编号", how="left")

In [24]:
df11 = pd.merge(df11, df51, left_on="订单编号", right_on="运营商单号", how="left")

In [25]:
df11 = pd.merge(df11, df52, left_on="订单编号", right_on="运营商单号", how="left")

In [26]:
df11['入网手机号'] = df11['入网手机号'].apply(int)

In [27]:
df11 = pd.merge(df11, df34[[ '用户名']],left_on="入网手机号", right_on="用户名", how="left") #'订单编号',

In [28]:
df11['派单'] = df11["是否下省"].apply(paidan)

In [29]:
df11['派卡'] = df11["物流单号_y"].apply(paidan)

In [30]:
df11['上门'] = df11["APP操作时间"].apply(paidan)

In [31]:
df11['首充'] = df11["订单编号"].apply(paidan) #订单编号_y

In [32]:
df11.columns

Index(['订单编号', '原始订单', '订单类型', '订单状态', '订单子状态', '取消原因', '订单挂起原因', '安装失败原因',
       '订单来源', '结算状态',
       ...
       '是否下省', '运营商单号_x', '物流单号_y', '运营商单号_y', 'APP操作时间', '用户名_y', '派单', '派卡',
       '上门', '首充'],
      dtype='object', length=110)

In [33]:
df11['来单量'] = df11['订单编号'].apply(laidan)

In [34]:
df11['发货量'] = df11.apply(lambda x: fahuo(x["物流单号_x"], x["订单状态"], x["物流签收时间"]), axis=1)

In [35]:
df11['签收量'] = df11.apply(lambda x: qianshou(x["物流签收时间"], x["订单状态"]), axis=1)

In [36]:
df11['激活量'] = df11["订单状态"].apply(jihuo)

In [37]:
df = df11[["号码归属地","销售品编号","营业厅送货方式","派单","派卡","上门","首充","发货量","签收量","激活量","来单量"]]

In [38]:
split1 = pd.DataFrame((x.split('/') for x in df['号码归属地']),index=df.index,columns=['所属省','所属市'])
df = pd.merge(df, split1, left_index=True, right_index=True)

In [39]:
df["销售品编号"] = df["销售品编号"].map(lambda x : str(x))

In [40]:
biaoka = pd.read_excel(inputPath2 + "产品标卡.xlsx")
biaoka["销售品编号"] = biaoka["销售品编号"].map(lambda x: str(x))

In [41]:
data_JM3 = pd.merge(df, biaoka, how="left", on="销售品编号")

In [42]:
df_JM3 = data_JM3.copy()

In [43]:
df_SHCH_product = pd.pivot_table(df_JM3, index=[u'分类'], values=['首充'],
                                      aggfunc=[np.sum], fill_value=0,
                                      margins=True, margins_name='合计')

In [49]:
df_JD = df_JM3[df_JM3["营业厅送货方式"] == "线下送货（京东模式）"]
df_JD_product = df_JD.groupby(by=["分类"]).sum()
df_JD_product.loc['合计'] = df_JD_product.apply(lambda x: x.sum())
df_JD_product["产品派单量占比"] = df_JD_product["派单"].map(
lambda x: (2 * x) / (sum(df_JD_product["派单"])+0.00001)
)
df_JD_product["派卡率"] = df_JD_product["派卡"] / (df_JD_product["派单"] + 0.00001)
df_JD_product["异常筛查"] = df_JD_product["派单"] - df_JD_product["派卡"]
df_JD_product["异常筛查率"] = df_JD_product["异常筛查"] / (df_JD_product["派单"] +0.00001)
df_JD_product["上门率"] = df_JD_product["上门"] / (df_JD_product["派卡"]+0.00001)
df_JD_product["未上门"] = df_JD_product["派卡"] - df_JD_product["上门"]
df_JD_product["未上门率"] = df_JD_product["未上门"] / (df_JD_product["派卡"]+0.00001)
df_JD_product["产品激活量占比"] = df_JD_product["激活量"].map(
lambda x: (2 * x) / (sum(df_JD_product["激活量"])+0.00001)
)
df_JD_product["激活率"] = df_JD_product["激活量"] / (df_JD_product["派卡"]+0.00001)
df_JD_product["产品首充量占比"] = df_JD_product["首充"].map(
lambda x: (2 * x) / (sum(df_JD_product["首充"])+0.00001)
)
df_JD_product["首充率"] = df_JD_product["首充"] / (df_JD_product["派卡"]+0.00001)
df_JD_product["全量首充"] = df_SHCH_product
df_JD_product = df_JD_product.reset_index('分类')
df_JD_product2 = df_JD_product[["分类","派单",
"产品派单量占比",
"派卡",
"派卡率",
"异常筛查",
"异常筛查率",
"上门",
"上门率",
"未上门",
"未上门率",
"激活量",
"产品激活量占比",
"激活率",
"首充",
"产品首充量占比",
"首充率",
"全量首充"]]

In [50]:
df_JD_product2.iloc[10,]

分类                 翼视卡
派单                   0
产品派单量占比              0
派卡                   0
派卡率                  0
异常筛查                 0
异常筛查率                0
上门                   0
上门率                  0
未上门                  0
未上门率                 0
激活量                  2
产品激活量占比    0.000184247
激活率             200000
首充                   5
产品首充量占比    0.000207503
首充率             500000
全量首充                 6
Name: 10, dtype: object